In [1]:
#!/usr/bin/env python3
"""
compare_abm_contacts.py

Load ABM (normal) and ABM30 temporal networks and compare:
  • Average degree (per-timestep mean; and aggregated network)
  • Inter-contact times (pooled across all dyads)
  • Burstiness of inter-contact times: B = (sigma - mu) / (sigma + mu)

Assumptions
-----------
- Temporal networks are lists of NetworkX graphs, one per timestep.
- Nodes that do not appear in a timestep are treated as present with degree 0
  when computing per-timestep average degree (we normalize by the global
  number of individuals).
- Inter-contact times are computed per undirected dyad (u, v) as gaps in
  timesteps between consecutive contacts.

Outputs
-------
- Prints a neat comparison table for ABM vs. ABM30.
"""

import os
import math
import statistics as stats
from typing import Dict, List, Tuple

from graph_utils import load_abm_network, get_individuals_from_graph


def load_temporal_from_csv(csv_path: str):
    """
    Wrapper around your existing loader with the same arguments
    you used in your other script.
    """
    return load_abm_network(
        file_path=csv_path,
        remap=True,
        node_attributes=None,
        required_ageGroup=None,
    )


def average_degree_timeseries(temporal: List) -> Tuple[float, float]:
    """
    Return (mean, std) of per-timestep average degree, using the GLOBAL
    node count so absent nodes count as degree 0.
    """
    n_nodes_global = len(get_individuals_from_graph(temporal))
    if n_nodes_global == 0:
        return float("nan"), float("nan")

    vals = []
    for G in temporal:
        m_t = G.number_of_edges()
        avg_deg_t = (2.0 * m_t) / n_nodes_global
        vals.append(avg_deg_t)

    mean = sum(vals) / len(vals) if vals else float("nan")
    std = stats.pstdev(vals) if len(vals) > 1 else 0.0
    return mean, std



def inter_contact_times(temporal: List) -> List[int]:
    """
    Compute pooled inter-contact times across all dyads.
    For each undirected edge, collect the gaps (in timesteps) between
    consecutive appearances.
    """
    last_seen = {}
    gaps = []

    for t, G in enumerate(temporal):
        for u, v in G.edges():
            if u == v:
                continue
            e = (u, v) if u < v else (v, u)
            if e in last_seen:
                dt = t - last_seen[e]
                if dt > 0:
                    gaps.append(dt)
            last_seen[e] = t

    return gaps


def burstiness(gaps: List[int]) -> float:
    """
    B = (sigma - mu) / (sigma + mu), with safeguards.
    Returns NaN if undefined (e.g., fewer than 2 gaps or denominator = 0).
    """
    if len(gaps) < 2:
        return float("nan")
    mu = stats.fmean(gaps)
    sigma = stats.pstdev(gaps)
    denom = sigma + mu
    if denom <= 0:
        return float("nan")
    return (sigma - mu) / denom


def summarize_network(label: str, temporal: List) -> Dict[str, float]:
    mean_deg_ts, std_deg_ts = average_degree_timeseries(temporal)
    gaps = inter_contact_times(temporal)
    B = burstiness(gaps)

    # Helpful gap stats
    gap_count = len(gaps)
    gap_mean = stats.fmean(gaps) if gap_count else float("nan")
    gap_median = stats.median(gaps) if gap_count else float("nan")
    gap_p90 = float("nan")
    if gap_count:
        sorted_g = sorted(gaps)
        idx = max(0, int(0.9 * (gap_count - 1)))
        gap_p90 = sorted_g[idx]

    return {
        "label": label,
        "timesteps": len(temporal),
        "nodes_global": len(get_individuals_from_graph(temporal)),
        "avg_degree_ts_mean": mean_deg_ts,
        "avg_degree_ts_std": std_deg_ts,
        "ict_count": gap_count,
        "ict_mean": gap_mean,
        "ict_median": gap_median,
        "ict_p90": gap_p90,
        "burstiness": B,
    }


def print_comparison(a: Dict[str, float], b: Dict[str, float]) -> None:
    def fmt(x):
        if isinstance(x, (int,)):
            return f"{x:d}"
        if x != x:  # NaN check
            return "NaN"
        return f"{x:.4f}"

    rows = [
        ("Timesteps", a["timesteps"], b["timesteps"]),
        ("Global nodes", a["nodes_global"], b["nodes_global"]),
        ("Avg degree (per-timestep mean)", a["avg_degree_ts_mean"], b["avg_degree_ts_mean"]),
        ("Avg degree (per-timestep std)", a["avg_degree_ts_std"], b["avg_degree_ts_std"]),
        ("Inter-contact times (mean)", a["ict_mean"], b["ict_mean"]),
        ("Inter-contact times (median)", a["ict_median"], b["ict_median"]),
        ("Inter-contact times (p90)", a["ict_p90"], b["ict_p90"]),
        ("Burstiness B", a["burstiness"], b["burstiness"]),
    ]

    label_a = a["label"]
    label_b = b["label"]

    print("\nComparison: ABM vs ABM30\n")
    colw = 38
    print(f"{'Metric':{colw}} | {label_a:^16} | {label_b:^16}")
    print("-" * (colw + 1 + 18 + 1 + 18))
    for name, va, vb in rows:
        print(f"{name:{colw}} | {fmt(va):>16} | {fmt(vb):>16}")
    print()





In [2]:

abm_csv =  "../data/micro_abm_contacts.csv"
abm30_csv = "../data/micro_abm_contacts30.csv"

print("Loading ABM (normal)…")
abm_temporal = load_temporal_from_csv(abm_csv)

print("Loading ABM30…")
abm30_temporal = load_temporal_from_csv(abm30_csv)

abm_summary = summarize_network("ABM", abm_temporal)
abm30_summary = summarize_network("ABM30", abm30_temporal)

print_comparison(abm_summary, abm30_summary)


Loading ABM (normal)…


Loading ABM network: 100%|██████████| 240/240 [00:06<00:00, 38.77it/s]


Loading ABM30…


Loading ABM network: 100%|██████████| 720/720 [00:21<00:00, 33.33it/s]



Comparison: ABM vs ABM30

Metric                                 |       ABM        |      ABM30      
----------------------------------------------------------------------------
Timesteps                              |              240 |              720
Global nodes                           |             2058 |             2058
Avg degree (per-timestep mean)         |           3.9997 |           3.8461
Avg degree (per-timestep std)          |           2.9858 |           2.8968
Inter-contact times (mean)             |           6.5523 |           7.6531
Inter-contact times (median)           |                1 |           1.0000
Inter-contact times (p90)              |               22 |               23
Burstiness B                           |           0.3217 |           0.4475

